# Study the impact of the news

In [1]:
import os
import sys
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from pyhanlp import *
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import matplotlib.pyplot as plt
from gensim import corpora
from collections import defaultdict
from snownlp import SnowNLP
from updatedb import UpdateDB
HanLP.Config.ShowTermNature = False
%matplotlib inline

### 1. 处理价格数据

In [2]:
price_all = pd.read_csv('../../data/wind/ashareeodprices.csv',dtype={'TRADE_DT':str}).sort_values('TRADE_DT')
price_all = price_all[price_all['TRADE_DT']>'20100101']
group_code = price_all.groupby('S_INFO_WINDCODE')
price_all['CloseRet_t-2_t+1'] = group_code['S_DQ_ADJCLOSE'].shift(-1) / group_code['S_DQ_ADJCLOSE'].shift(2)
price_all['OpenRet_t-1_t+1'] = group_code['S_DQ_ADJOPEN'].shift(-1) / group_code['S_DQ_ADJOPEN'].shift(1)
price_all['OpenRet_t_t+1'] = group_code['S_DQ_ADJOPEN'].shift(-1) / group_code['S_DQ_ADJOPEN'].shift(0)

price_all.head()
# pp = []
# group_date = price_all.groupby('TRADE_DT')
# for g in group_date.groups:
#     gd = group_date.get_group(g).sort_values('CloseRet_t-2_t+1')
#     gd = gd[~gd['CloseRet_t-2_t+1'].isnull()]
#     gd['p'] = np.arange(len(gd)) / len(gd)
#     pp.append(gd)
# pp = pd.concat(pp)
# pp

,S_INFO_WINDCODE,TRADE_DT,CRNCY_CODE,S_DQ_PRECLOSE,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_CHANGE,S_DQ_PCTCHANGE,...,S_DQ_ADJLOW,S_DQ_ADJCLOSE,S_DQ_ADJFACTOR,S_DQ_AVGPRICE,S_DQ_TRADESTATUS,OPDATE,OPMODE,CloseRet_t-2_t+1,OpenRet_t-1_t+1,OpenRet_t_t+1
3985486,000711.SZ,20100104,CNY,10.11,10.08,10.23,10.08,10.12,0.01,0.0989,...,27.23,27.34,2.701469,10.1398,交易,2017-11-28 13:20:33,1,NaN,NaN,1.004040
3985485,000690.SZ,20100104,CNY,9.66,9.71,9.75,9.57,9.68,0.02,0.2070,...,212.75,215.20,22.230928,9.6279,交易,2017-11-28 13:20:33,1,NaN,NaN,0.995877
3985484,000685.SZ,20100104,CNY,30.47,30.47,30.47,30.47,30.47,0.00,0.0000,...,159.15,159.15,5.223270,30.4700,停牌,2017-11-28 13:20:33,1,NaN,NaN,1.000000
3985483,000761.SZ,20100104,CNY,7.07,7.10,7.16,6.99,7.00,-0.07,-0.9901,...,12.24,12.25,1.750423,7.0483,交易,2017-11-28 13:20:33,1,NaN,NaN,0.987128
3985482,000792.SZ,20100104,CNY,56.99,57.25,60.17,57.25,59.51,2.52,4.4218,...,269.68,280.33,4.710616,58.9180,交易,2017-11-28 13:20:33,1,NaN,NaN,1.040196


In [3]:
updb = UpdateDB('10.24.224.249','fineng','123456')
index = updb.pull_index(['000300.SH']).loc['20100101':].reset_index().rename(columns={'TRADE_DT':'date'}).sort_values('date')
index['next_date'] = index['date'].shift(-1)
index.head()

S_INFO_WINDCODE,date,000300.SH,next_date
0,20100104,3535.229,20100105
1,20100105,3564.038,20100106
2,20100106,3541.727,20100107
3,20100107,3471.456,20100108
4,20100108,3480.130,20100111


In [4]:
code_name = updb.pull_data_accor_date('MyAShareDescription','wind',set_index=False).set_index('S_INFO_WINDCODE')
code_name

,S_INFO_NAME
S_INFO_WINDCODE,
000001.SZ,平安银行
000002.SZ,万科A
000003.SZ,PT金田A(退市)
000004.SZ,国农科技
000005.SZ,世纪星源
000006.SZ,深振业A
000007.SZ,全新好
000008.SZ,神州高铁
000009.SZ,中国宝安


## 2. 处理新闻数据

In [5]:
newsfiles = os.listdir('../news')
newsfiles = list(filter(lambda x: x.split('.')[1][0]=='S', newsfiles))
newsfiles

['002551.SZ.csv',
 '000935.SZ.csv',
 '603798.SH.csv',
 '600518.SH.csv',
 '300515.SZ.csv',
 '300274.SZ.csv',
 '603567.SH.csv',
 '002030.SZ.csv',
 '603800.SH.csv',
 '300038.SZ.csv',
 '000597.SZ.csv',
 '300167.SZ.csv',
 '601811.SH.csv',
 '600507.SH.csv',
 '603638.SH.csv',
 '603181.SH.csv',
 '300476.SZ.csv',
 '300341.SZ.csv',
 '002438.SZ.csv',
 '300470.SZ.csv',
 '300517.SZ.csv',
 '300406.SZ.csv',
 '603309.SH.csv',
 '603660.SH.csv',
 '300044.SZ.csv',
 '002309.SZ.csv',
 '300622.SZ.csv',
 '300391.SZ.csv',
 '300053.SZ.csv',
 '300117.SZ.csv',
 '600038.SH.csv',
 '600589.SH.csv',
 '601388.SH.csv',
 '600653.SH.csv',
 '600526.SH.csv',
 '300711.SZ.csv',
 '000858.SZ.csv',
 '002387.SZ.csv',
 '300737.SZ.csv',
 '300007.SZ.csv',
 '600289.SH.csv',
 '601216.SH.csv',
 '300400.SZ.csv',
 '600774.SH.csv',
 '603908.SH.csv',
 '000987.SZ.csv',
 '300636.SZ.csv',
 '603650.SH.csv',
 '600798.SH.csv',
 '603321.SH.csv',
 '300556.SZ.csv',
 '000657.SZ.csv',
 '600648.SH.csv',
 '600012.SH.csv',
 '600877.SH.csv',
 '000823.S

### 3. 停词表中加入公司名

In [13]:
def process(symbol,first=False):

    # process news
    news = pd.read_csv('../news/%s.csv' % symbol,dtype={'date':str})[['date','title','time','content']]
    news = news.dropna()
    news = news.sort_values('date')
    nonsense=['融资融券信息','大宗交易','今日超大单流','龙虎榜','下跌','上涨','跌幅','涨幅','涨停','跌停'
             '大涨','大跌','跳水','盘中','融资余额','反弹','回调','火箭发射']
    nonsense = '|'.join(nonsense)
    is_useful = np.array([not bool(re.search(nonsense,l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('大宗交易',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('今日超大单流',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('龙虎榜',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('下跌',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('上涨',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('跌幅',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('涨幅',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('涨停',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('大涨',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('大跌',l)) for l in news['title'].values])

#     is_useful = is_useful & np.array([not bool(re.search('',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('跳水',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('盘中',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('融资余额',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('反弹',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('回调',l)) for l in news['title'].values])
#     is_useful = is_useful & np.array([not bool(re.search('火箭发射',l)) for l in news['title'].values])

    news = news[is_useful]
    # news['content'] = news['title'] + news['content']
    news = news.drop('content',axis=1)
    news['date'] = list(map(lambda x:x.replace('-',''), news['date'].values))

    news = news.merge(index[['date','next_date']],how='left')
    news.loc[news['time']>'13:00:00','date'] = np.nan
    news['date'] = news['date'].fillna(news['next_date'])

    price = price_all[price_all['S_INFO_WINDCODE']==symbol][['TRADE_DT','CloseRet_t-2_t+1']].rename(columns={'TRADE_DT':'date'})
    news = news.merge(price)

    pos_news = news.loc[news['CloseRet_t-2_t+1']>1.1,['title']]
    pos_news['label'] = 1
    neg_news = news.loc[news['CloseRet_t-2_t+1']<0.9,['title']]
    neg_news['label'] = 0
    ok_news = pd.concat([pos_news,neg_news])[['label','title']]
    ok_news['title'] = [s.replace(code_name.loc[symbol].values[0],'') for s in ok_news['title']]
    print(ok_news)
#     if first:
#         ok_news[['label','title']].to_csv('news.csv', index=False)
#     else:
#         ok_news[['label','title']].to_csv('news.csv', mode='a',header=False, index=False)


nf = newsfiles[0]
process(nf[:9],first=True)
for i,nf in enumerate(newsfiles[1:]):
    if (len(nf)==13):
        process(nf[:9],first=False)
        print('processing %s %d/%d' % (nf[:9], i, len(newsfiles)))


     label                   title
0        1               承建2亿元医院项目
13       1           签署2.1亿合作框架协议书
14       1             获2.1亿元意向性合同
26       1                 中标7亿元大单
27       1          签订1.2亿元项目合作协议书
28       1   31亿订单来自何方 公关地方政府模式被质疑
47       1        中标医院迁建项目 金额2.5亿元
48       1         预中标2.5亿医院整体搬迁项目
49       1       预中标2.5亿项目 对业绩贡献积极
75       1             拟前海及香港设立子公司
121      1           ：将介入医院投资运营和管理
122      1   :2015年5月14日投资者关系活动记录表
123      1     全资子公司参与公立医院改革 20日复牌
124      1                 投资秦皇岛二院
135      1              签1亿元医院建设合同
136      1           子公司签3亿元医院建设合同
137      1           股东拟增持已发股份总额1%
159      1         确认中标2.5亿元县级医院项目
160      1          2.5亿中标招商引资建设项目
165      1           拟“联手”兰州新区管理委员
36       0     模式：“买方信贷”掩盖下的“变异BT”
37       0               大订单驱动业绩成长
42       0              业绩增长难阻小非减持
77       0           遭三股东合计减持550万股
78       0          遭董事长等减持共计550万股
130      0                 签医院迁建项目
142      0        拟斥资2亿合资设医疗产业并购基金
157      0  :2015年12

   label                title
0      1     借互联网搭建产业新模式 的新契机
1      1  健康投资两手抓 药业站在下一个“风口”
processing 603567.SH 5/3668
     label                      title
18       1            及其子公司获三个医疗器械注册证
24       1                ：大股东增持 彰显信心
25       1            东兴证券：维持“强烈推荐”评级
42       1       ：2014年4月3日投资者关系活动记录表
43       1           ：2014年4月3日调研活动附件
44       1       ：2014年4月9日投资者关系活动记录表
45       1           ：2014年4月9日调研活动附件
72       1            两股东将489万股卖出回购融资
73       1               创历史新高 机构逢高减持
136      1          去年净利1.51亿 同比增4.9%
138      1         利增6.5% 拟10送2派0.23元
146      1      :2015年5月18日投资者关系活动记录表
147      1          :2015年5月18日调研活动附件
148      1              ：有中东呼吸综合症检测试剂
149      1            ：MERS检测剂系科研用未临床
150      1                      创历史新高
162      1            ：PCR诊断试剂增速保持20%
185      1     基因测序开启精准医疗蓝海 10只个股值得期待
206      1    华大基因闯关创业板 基因测序概念或闻“基”起舞
207      1   华大基因闯关创业板 A股相关概念股或闻“基”起舞
218      1           基因测序迎机遇 7股暗藏暴涨因子
229      1  健康产业领袖峰会召开在即 两大逻辑布局大健康概念股
282     

Empty DataFrame
Columns: [label, title]
Index: []
processing 601811.SH 11/3668
     label                             title
22       1                           拟10派10元
24       1              13亿分红吃掉3年家底 大股东独享近7成
27       1              拟竞拍凯盛矿业49%股权 底价3.6亿元
28       1                      拟竞购玉石矿商49%股权
117      1              去年净利同比增长1.16% 拟10派8元
118      1                    净利增1% 拟每10股派8元
125      1                    拟收购特种汽车公司60%股权
126      1               收购特种汽车60%股权 拓宽公司产业链
127      1            (600507)高管持股数据一览(6-10)
128      1            拟实施10派8元 多元化经营开辟新经济增长点
129      1                   ：2014年度分红派息实施公告
130      1            (600507)高管持股数据一览(6-12)
135      1                新三板挂牌券商东海证券上半年净利居首
139      1                  跨界竞购海际证券 对券商情有独钟
140      1                   拟竞购海际证券66.67%股权
141      1                    :汇金证金及多个资管计划持股
142      1                  前3季盈1.6亿元 同比降64%
143      1              前三季度实现净利1.6亿元 同比降逾六成
166      1                     钢铁板块表现抢眼 涨逾6%
183      1           

     label                                           title
30       1                           中利科技获签日本800MW光伏电站开发协议
34       1                            中利科技子公司出售300MW地面光伏项目
35       1                            中利科技子公司出售300MW光伏电站项目
40       1                                       三机构买入中利科技
102      1                              中利科技获公司五位高管增持100万股
103      1                             中利科技五高管合计增持100.82万股
104      1                      中利科技(002309)高管持股数据一览(3-17)
105      1                      中利科技(002309)高管持股数据一览(3-18)
125      1                               中利腾晖总价不超30亿甩卖发电项目
126      1                               中利科技实控人拟至多溢价超六成增持
127      1                                中利科技实控人拟半年增持超千万股
112      0                        中利科技拟募资25亿投建光伏电站项目 21日复牌
113      0                            中利科技拟定增募资25亿投建光伏电站项目
114      0                              中利科技募资25亿建光伏电站 明复牌
116      0                              中利科技五位高管合计共增持182万股
117      0                      中利科技(002309)高管持股数据一览(7-2

    label                      title
3       1       拟购32亿元资产 哈飞股份“包乘”直升机
4       1            哈飞股份获33亿直升机资产注入
5       1             哈飞股份重大资产重组疑团待解
10      1           哈飞股份借重组坐稳直升机龙头地位
31      1          基金抢筹军工股 6亿元认购哈飞股份
59      1     (600038)高管持股数据一览(7-10)
60      1            千项成果将展出 军工行情动力足
61      1     千项重大成果将亮相军民融合展 军工行情动力足
75      1  近40亿资金抢筹军工股 未来10年行业景气度或高涨
76      1               军工股崛起 涨8.11%
63      0       国防军工：下半年是最佳投资时机 推荐9股
64      0                  中信建投：买入评级
65      0                通用航空领跌 跌近8%
71      0          军工：万亿盛宴大幕开启 推荐22股
processing 600038.SH 29/3668
    label                               title
23      1                      按原计划12亿元收购森华易腾
24      1                       终止重组调整 按原方案推进
25      1                   终止调整重组方案 继续按原方案推进
30      1                           中银国际：买入评级
60      1                       ：意向收购数据中心企业股权
61      1           加码大数据 广州榕泰拟购能通科技20%-51%股份
62      1                       前后公告自相矛盾 想蹭5G
63      1                  ：内容分发网络业务的收入占比还比较小
64

...

     label                              title
9        1                  获100万政府补贴 确认为递延收益
25       1               拟发行股份收购亿金环保53%股权 明复牌
26       1                   拟1.44亿元揽入亿金环保控股权
27       1                业绩下滑谋求转型 1.4亿控股亿金环保
28       1             揽亿金环保53%股权 “多方捆绑”设计成亮点
29       1             1.44亿收购牵出神秘第三方 疑配合PE退出
47       1                          收购案公告暗藏玄机
82       1               :2015年5月5日投资者关系活动记录表
96       1                   员工持股计划已买入57.22万股
98       1                  ：实际控制人违法减持 罚款90万元
99       1                   实际控制人违法减持 罚款90万元
100      1                     大股东因违规减持被罚款90万
107      1                    推进绿色产业新项目扭转机构预期
126      1                      29股每笔成交量增超50%
133      1                      预计上半年业绩同比扭亏为盈
134      1          (300249)高管相关人员持股变动信息(7-4)
138      1             上半年实现扭亏为盈 斥资2000万增资子公司
53       0             (300249)高管持股数据一览(12-5)
88       0                      业绩骤降重组告吹 承压跌停
89       0         沪指全周收跌11.54% 418股单周股价缩水逾两成
90       0             (300249)高管持

   label                    title
9      1  股东和邦集团补充质押1000万股 用于补充质押
2      0                 2月5日打开跌停
4      0                 7月4日打开跌停
6      0               10月18日打开跌停
processing 603477.SH 667/3668
    label                                      title
0       1                           遭开板 上市至今股价涨超300%
21      1                     (300385)高管持股数据一览(7-15)
25      1                      (300385)高管持股数据一览(8-3)
28      1                     (300385)高管持股数据一览(8-25)
38      1                                  东吴证券：买入评级
20      0                       (300385)解禁股信息(06-26)
22      0                     (300385)高管持股数据一览(7-24)
70      0       三季报速递：2017年前三季度盈利4450.79万元，同比增长9.88%
71      0  预计2017年净利润5411.15万元至7983.65万元 同比下降39%至10%
processing 300385.SZ 668/3668
     label                         title
2        1         (002740)高管持股数据一览(4-3)
19       1    游族网络拟10转20派1.8元 多家公司发布业绩快报
20       1                     年报拟推10转20
21       1             业绩未见起色拟10转20派1.1元
22       1             预披露高送转 10转20

     label                         title
55       1              彩虹精化正在积极推进股权激励计划
96       1       彩虹精化与兴业太阳能就光伏电站开发达成战略合作
97       1              彩虹精化与兴业太阳能开发光伏电站
98       1            彩虹精化与兴业太阳能合作项目额达16
99       1                彩虹精化：签署战略合作协议书
134      1            彩虹精化定增20.6亿神秘人王军现身
135      1           彩虹精化拟定增募资20.6亿 2日复牌
136      1                 彩虹精化拟募资加码光伏发电
137      1              彩虹精化：增发加码分布式光伏发电
138      1         彩虹精化去年利减30% 拟10股派0.6元
143      1     彩虹精化:2015年3月20日投资者关系活动记录表
144      1     彩虹精化:2015年3月27日投资者关系活动记录表
145      1         彩虹精化与核建二二公司签署80亿元合作协议
146      1     彩虹精化:2015年3月26日投资者关系活动记录表
147      1        彩虹精化与核建二二公司签订80亿光伏合作协议
148      1     彩虹精化:2015年3月31日投资者关系活动记录表
171      1             彩虹精化拟2.2亿收购光伏发电公司
172      1                 齐鲁证券：彩虹精化买入评级
173      1              彩虹精化与招商局资本合资光伏基金
174      1    彩虹精化设50亿光伏基金 在建及储备项目超600兆瓦
175      1    彩虹精化设50亿光伏基金 在建及储备项目超600MW
179      1            彩虹精化停止筹划收购事项 18日复牌
184      1            彩虹精化全资子公司收到5200万补助
185      1    彩虹

    label                     title
6       1            长航油运上半年业绩亏损逾3亿
68      1              退市长油大股东股份被冻结
69      1   退市长油上演“末日疯狂” 投资者需注意控制风险
33      0       证监会：ST长油符合条件可申请重新上市
34      0      *ST长油更名退市长油 成两市最低价股票
35      0       *ST长油开启“交易三十日退市”倒计时
36      0       退市长油一字跌停 近5亿资金跌停板甩卖
37      0      退市长油复牌首日一字跌停 投资者组团喊停
38      0       4亿资金封死跌停 退市长油跌停之旅开启
39      0         退市长油直奔跌停 最快本周五成仙股
40      0  退市长油进入整理期：4亿股跌停单 最快周五成仙股
41      0          还买退市长油 谁在幻想奇迹出现？
42      0         退市长油连续3跌停 最快周五成仙股
43      0          买入退市长油 谁在幻想奇迹出现？
44      0      退市长油连续4跌停 仍有11亿元贷款逾期
45      0              退市长油11亿元贷款逾期
46      0      退市长油或成仙股 仍有投机客“刀口舔血”
47      0   退市长油连续5跌停 成A股史上首家央企“仙股”
48      0    退市长油股价跌穿1元 成A股首家央企“仙股”
49      0   退市长油跌破1元成首家央企仙股 多股或步其后尘
50      0      退市长油或成仙股 投机客疯狂买入期待奇迹
51      0   退市长油五连跌后成交额放大 游资豪赌仙股演神话
52      0           “退市长油”成A股首家央企仙股
53      0                 退市长油打开跌停板
54      0          退市长油现豪赌 2亿资金抄底被套
55      0     退市长油开板引跟风盘 游资两亿豪赌仍挨跌停
56      0           退市长油7跌停 

KeyboardInterrupt: 

In [12]:
bool(re.search('哈哈|呀呀','你很大哈哈呀'))

True

In [21]:
newsfiles[1]

'000935.SZ.csv'

In [23]:
process(newsfiles[1][:9])

     label                               title
35       1                  ：拉法基中国赠送股份登记日6月21日
36       1                            21日起实施赠股
56       1                向大股东发行股份收购拉法基水泥25%股权
57       1                重启收购都江堰拉法基股权 估值缩水近8%
58       1              定增募8.32亿 拟购都江堰拉法基25%股权
59       1               拟定增8.3亿元收购都江堰拉法基25%股权
60       1             拟定增1.48亿股购都江堰拉法基水泥25%股权
61       1                    时隔九个月 重启收购都江堰拉法基
62       1                      大股东践诺 定增再购水泥资产
63       1                     定增再购都江堰拉法基25%股权
78       1                            发行3亿中期票据
104      1            水泥今年或面临全行业亏损 房地产回暖对其影响微弱
116      1             解决同业竞争致力产业转型 股份转让有望实现多赢
117      1                           “换主”后或将重组
120      1               和谐恒源及天津赛克环拟要约收购2.41亿股
121      1               实控人生变 IDG合伙人现身称择机注入资产
122      1                  新股东履行要约收购义务 重组预期升温
123      1                        重组预期升温创逾8年新高
124      1                 A股市场的“们”：实控人变更造“妖股”
125      1                            14日起停牌自查
126      1   

In [24]:
symbol = newsfiles[1][:9]
nonsense=['融资融券信息','大宗交易','今日超大单流','龙虎榜','下跌','上涨','跌幅','涨幅','涨停','跌停'
             '大涨','大跌','跳水','盘中','融资余额','反弹','回调','火箭发射']
nonsense = '|'.join(nonsense)
news = pd.read_csv('../news/%s.csv' % symbol,dtype={'date':str})[['date','title','time','content']]
news = news.dropna()
news = news.sort_values('date')
is_useful = np.array([not bool(re.search(nonsense,l)) for l in news['title'].values])
is_useful

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True, False,  True,  True, False,  True,  True,
        True,  True,